## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-10-18-48-21 +0000,bbc,Phillipson cements status as Labour deputy fro...,https://www.bbc.com/news/articles/c5yvdxvx449o...
1,2025-09-10-18-46-35 +0000,nypost,Lions eat zookeeper alive in horrific attack f...,https://nypost.com/2025/09/10/world-news/lions...
2,2025-09-10-18-44-03 +0000,cbc,Canada 'evaluating' relationship with Israel a...,https://www.cbc.ca/news/politics/canada-evalua...
3,2025-09-10-18-38-00 +0000,wsj,NATO Planes Shoot Down Russian Drones Deep Ins...,https://www.wsj.com/world/europe/poland-russia...
4,2025-09-10-18-36-30 +0000,startribune,Live: Teamsters strike at the University of Mi...,https://www.startribune.com/live-hundreds-of-u...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2298/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
117,trump,62
66,new,24
18,israel,23
19,qatar,19
97,will,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
309,2025-09-09-22-19-57 +0000,wapo,Israeli strike targeting Hamas in Qatar ‘does ...,https://www.washingtonpost.com/world/2025/09/0...,153
76,2025-09-10-16-40-00 +0000,wsj,Nearly half a day after NATO ordered jet fight...,https://www.wsj.com/politics/national-security...,139
148,2025-09-10-13-37-45 +0000,cbc,Supreme Court will hear challenge to Trump's t...,https://www.cbc.ca/news/world/scotus-tariffs-i...,122
266,2025-09-10-01-14-36 +0000,nyt,"With Qatar Attack, Israel Again Leaves Trump i...",https://www.nytimes.com/2025/09/09/us/politics...,120
251,2025-09-10-02-45-20 +0000,nypost,Court rules that Lisa Cook can keep job as Fed...,https://nypost.com/2025/09/09/us-news/court-ru...,119


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
309,153,2025-09-09-22-19-57 +0000,wapo,Israeli strike targeting Hamas in Qatar ‘does ...,https://www.washingtonpost.com/world/2025/09/0...
57,116,2025-09-10-17-08-00 +0000,wsj,Former Vice President Kamala Harris said in a ...,https://www.wsj.com/politics/elections/harris-...
76,64,2025-09-10-16-40-00 +0000,wsj,Nearly half a day after NATO ordered jet fight...,https://www.wsj.com/politics/national-security...
148,60,2025-09-10-13-37-45 +0000,cbc,Supreme Court will hear challenge to Trump's t...,https://www.cbc.ca/news/world/scotus-tariffs-i...
193,59,2025-09-10-10-00-00 +0000,nypost,Lisa Cook should be kicked out of Fed’s Septem...,https://nypost.com/2025/09/10/us-news/lisa-coo...
274,48,2025-09-10-00-36-34 +0000,nypost,Poland’s airspace ‘repeatedly violated’ by Rus...,https://nypost.com/2025/09/09/world-news/ukrai...
221,42,2025-09-10-08-17-33 +0000,nypost,South Korea sends plane to US to bring back wo...,https://nypost.com/2025/09/10/world-news/south...
273,40,2025-09-10-00-45-35 +0000,nyt,Macron Picks Sébastien Lecornu as France’s Nex...,https://www.nytimes.com/2025/09/09/world/europ...
349,39,2025-09-09-20-54-54 +0000,nypost,Wife of late Sen. John McCain facing calls to ...,https://nypost.com/2025/09/09/us-news/cindy-mc...
88,38,2025-09-10-16-15-28 +0000,bbc,"Nepal protests 'hijacked', Gen Z claim, as arm...",https://www.bbc.com/news/articles/cjd1ndmrej0o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
